In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv("portfolio_daily_returns.csv", parse_dates=['Date'], index_col='Date')
df.columns = ['Return']
returns = df['Return']

In [14]:
# Rolling 60-day CVaR
confidence_level = 0.95
def rolling_cvar(series, window=60):
    return series.rolling(window).apply(lambda x: x[x <= np.percentile(x, (1-confidence_level)*100)].mean())

df['Rolling_CVaR'] = rolling_cvar(returns)


In [15]:
# Cumulative Return
df['Cumulative_Return'] =(1 + returns).cumprod()

In [16]:
# Max Drawdown
df['Cumulative_Max'] = df['Cumulative_Return'].cummax()
df['Drawdown'] = df['Cumulative_Return'] / df['Cumulative_Max'] -1
df['Max_Drawdown'] = df['Drawdown'].cummin()

In [17]:
# Save time-series file for power BI
df_to_export = df[['Return', 'Rolling_CVaR', 'Cumulative_Return', 'Max_Drawdown']]
df_to_export.to_csv(r"C:\Users\amalm\OneDrive\Desktop\finamcial_data_analysis_lerning\project\value_at_risk\powerbi_data\powerbi_timeseries.csv")

In [20]:
# Summary metrics
var_95 = np.percentile(returns, 5)
cvar_95 = returns[returns <= var_95].mean()
sharpe = returns.mean() / returns.std()
sortino = returns.mean() / returns[returns < 0].std()
max_drawdown = df['Max_Drawdown'].min()

In [21]:
summary = pd.DataFrame({
    'Metric' : ['Historical VaR(95%)','CVaR(95%)', 'Max Drawdown', 'sharpe Ratio', 'Sortino Ratio'],
    'value': [f"{var_95:.4%}", f"{cvar_95:.4%}", f"{max_drawdown:.2%}", f"{sharpe:.4f}", f"{sortino:.4f}"]
})
summary.to_csv("powerbi_summary.csv", index=False)